In [1]:
!pip install POT

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.6/865.6 kB 22.3 MB/s eta 0:00:00


In [5]:
import numpy as np
import math
import matplotlib.pyplot as plt
import ot

In [ ]:
def ndot(a,b):
    return a[0]*b[0]-a[1]*b[1]

In [12]:
def clamp(value, min_val, max_val):
    return max(min_val, min(value, max_val))

In [3]:
def sdf_circle(x, r=0.5):
    return np.sqrt(np.sum(x**2, axis=1)) - r

In [4]:
def sdf_box(x, b=[0.7,0.7]):
    d = np.abs(x)-b
    max_d = np.maximum(d, 0.0)
    length_d = np.sqrt(np.sum(max_d**2, 1))
    return length_d + np.minimum(np.maximum(d[:,0], d[:,1]), 0.0)

In [15]:
def sdf_triangle(p,r=0.5):
    pi=np.sqrt(3.0)
    p[0]=np.abs(p[0])-r
    p[1]=p[1]+r/pi
    if(p[0]+pi*p[1]>0.0):
      p=(p[0]-pi*p[1],-pi*p[0]-p[1])/2.0;
    p[0]-=clamp(p[0],-2.0*r,0.0);
    max_p = np.maximum(p, 0.0)
    length_p = np.sqrt(np.sum(max_p**2, 1))
    return -np.linalg.norm(p)*np.sign(p[1])

In [17]:
def sdf_pentagon(p,r=0.5):
  k=np.array[0.809016994,0.587785252,0.726542528];
  p.x=np.abs(p[0])
  l=(-k[0],k[1])
  m=(k[0],k[1])
  p-=2.0*min(np.dot(l,p),0.0)*l;
  p-=2.0*min(np.dot(m,p),0.0)*m
  p-=(clamp(p[0],-r*k[2],r*k[2]),r);
  max_p = np.maximum(p, 0.0)
  length_p = np.sqrt(np.sum(max_p**2, 1))
  return -np.linalg.norm(p)*math.sin(p[1])

In [1]:
def sdf_hexagram(p,r=0.5):
    k =np.array[-0.5,0.8660254038,0.5773502692,1.7320508076]
    p=np.abs(p)
    p-=2.0*min(np.dot(k[:2],p),0.0)*k[:2]
    p-=2.0*min(np.dot(k[1::-1],p),0.0)*k[1::-1]
    p-=np.array([clamp(p[0],r*k[2]),r*k[3],r])
    return np.linalg.norm(p)*np.sign(p[1])

In [ ]:
def sdf_hexagon(p,r=0.5):
    k=np.array[-0.866025404,0.5,0.577350269]
    p=np.abs(p)
    p-=2.0*min(np.dot(k[:2]),p)*k[:2]
    p-=np.array([clamp(p[0],-r*k[2],r*k[2]),r])
    return np.linalg.norm(p)*np.sign(p[1])

In [ ]:
def sdf_rhombus(p,b):
    p=np.abs(p)
    h=clamp(ndot(b-2.0*p,b)/np.dot(b,b),-1.0,1.0)
    d=np.linalg.norm(p - 0.5 * b * np.array([1.0 - h, 1.0 + h]))
    return d*np.sign(p[0]*b[1]+p[1]*b[0]-b[0]*b[1])

In [ ]:
def sdf_star(p,r=0.5,rf=0.5):
    k1=np.array[0.809016994375, -0.587785252292]
    k2=np.array[-k1[0],k1[1]]
    p[0]=np.abs(p[0])
    p-=2.0*max(np.dot(k1,p),0.0)*k1
    p-=2.0*max(np.dot(k2,p),0.0)*k2
    p[0]=np.abs(p[0])
    p[1]-=r
    ba=rf*np.array([-k1[1],k1[0]])-np.array([0,1])
    h=clamp(np.dot(p,ba)/dot(ba,ba),0.0,r)
    return np.linalg.norm(p-ba*h)*np.sign(p[1]*ba[0]-p[0]*ba[1])

In [ ]:
def sdf_moon(p,d=0.5,ra=0.5,rb=0.5):
    p[1]=np.abs(p[1])
    a=(ra*ra-rb*rb+d*d)/(2.0*d)
    b=np.sqrt(max(ra*ra-a*a,0.0))
    if(d*(p[0]*b-p[1]*a)>d*d*max(b-p[1],0.0)):
        return np.linalg.norm(p-np.array([a,b]))
    return max(np.linalg.norm(p)-ra,(np.linalg.norm(p-np.array([d,0]))-rb))